In [131]:
from datetime import date, datetime, timedelta, time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [220]:
df = pd.read_csv("../taxi_data/predictions.csv", index_col=[0], parse_dates=["dt"])

In [225]:
aggr = pd.read_csv("../taxi_data/aggr_taxi_data.csv", 
                   index_col=["tpep_pickup_datetime"], 
                   parse_dates=["tpep_pickup_datetime"])

In [226]:
aggr = aggr.resample("H")
aggr = aggr.sum()

In [227]:
aggr.head(2)

,2,4,7,8,9,10,11,12,13,14,...,252,253,255,256,257,258,260,261,262,263
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0,49.0,24.0,0.0,0.0,2.0,2.0,4.0,63.0,3.0,...,0.0,0.0,24.0,17.0,2.0,0.0,10.0,54.0,129.0,361.0
2019-01-01 01:00:00,0.0,92.0,59.0,1.0,0.0,1.0,0.0,3.0,52.0,3.0,...,0.0,0.0,70.0,76.0,2.0,0.0,19.0,57.0,136.0,448.0


In [291]:
start = datetime(2019,9,10)
end = datetime(2019,9,15)
pu_zone = 236

df1 = df[(df.dt >= start) & (df.dt < end + timedelta(days=1))]
df2 = df1[df1.PULocationID==pu_zone]

In [292]:
df2.head(2)

,PULocationID,year,month,day,hour,wday,holiday,trips,predicted_trips,dt
75358,236,2019,9,10,0,1,0,22,40,2019-09-10 00:00:00
75359,236,2019,9,10,1,1,0,10,15,2019-09-10 01:00:00


In [293]:
df2 = df2.set_index("dt")

In [294]:
new_index = [df2.index[0]]
while new_index[-1] < df2.index[-1]:
    new_index.append(new_index[-1] + timedelta(hours=1))

In [295]:
df2 = df2.reindex(new_index)

In [296]:
df2 = df2.fillna(method='ffill')

In [297]:
start1 = start - timedelta(days=7)
end1 = end - timedelta(days=6)
df2["naive"] = aggr.loc[start1:end1, str(pu_zone)][:-1].tolist()

In [298]:
df2 = df2.reset_index()

In [299]:
import plotly.graph_objects as go
years = df2.dt
fig = go.Figure()
fig.add_trace(go.Bar(x=years,
                y = df2.trips,
                name='Actual',
                #marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=years,
                y = df2.predicted_trips,
                name='Prediction',
                #marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Bar(x=years,
                y = df2.naive,
                name='Naive Prediction',
                #marker_color='rgb(26, 118, 255)'
                ))
fig.update_layout(
    title='',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Rides',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(0, 0, 0, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    xaxis_showgrid=False,
    yaxis_showgrid=False
)
fig.show()